In [143]:
%run -n main.py
%matplotlib inline

In [286]:
data = read_data()

In [134]:
data.head()

,name,title,dosage,amount,price,id,date,code1,code2
0,None,Кагоцел,таблетки 12 мг 10 шт. - упаковки ячейковые контурные - пачки картонные,10.0,164.98,Р N002027/01,2010-10-03T04:00:00.000000000+0400,1808-Пр/10,4.60534000031e+12
1,None,Пирфотех 99mТc,лиофилизат для приготовления раствора для внутривенного введения - флаконы 10 мл (5) - коробки картонные,5.0,1435.32,Р N000494/01,2010-03-03T03:00:00.000000000+0300,1620-Пр/10,4.60622300006e+12
2,None,Технефит 99mTc,лиофилизат для приготовления раствора для внутривенного введения - флаконы 10 мл (5) - пачки картонные,5.0,1617.84,ЛС-002365,2010-03-03T03:00:00.000000000+0300,1620-Пр/10,4.60622300002e+12
3,None,Технефор 99mTc,лиофилизат для приготовления раствора для внутривенного введения - флаконы 10 мл (5) - коробки картонные,5.0,1662.12,ЛС-002366,2010-03-03T03:00:00.000000000+0300,1620-Пр/10,4.60622300002e+12
4,Абакавир,Зиаген,раствор для приема внутрь 20 мг/мл 240 мл - флаконы полиэтиленовые /в комплекте с адаптером и шприцем дозирующим/ - пачки картонные,1.0,2440.97,П N011612/02,2010-09-06T04:00:00.000000000+0400,5344-Пр/10,4.60700813069e+12


In [373]:
data[data.title.str.contains(u'ангин')]

,name,title,dosage,amount,price,id,date,code1,code2
1347,Калия и магния аспарагинат,Аспангин,таблетки покрытые оболочкой 50 шт. - банки полимерные - пачки картонные,50.0,30.24,Р N003316/02,2010-02-04T03:00:00.000000000+0300,2756-Пр/10,4.60377900987e+12
1375,Калия и магния аспарагинат,Панангин,раствор для внутривенного введения 10 мл - ампулы (5) - упаковки контурные пластиковые (поддоны) - пачки картонные,5.0,108.52,П N013093/01,2010-03-22T03:00:00.000000000+0300,2288-Пр/10,5.99700139152e+12
1376,Калия и магния аспарагинат,Панангин,таблетки покрытые пленочной оболочкой 50 шт. - флаконы полипропиленовые - пачки картонные,50.0,100.83,П N013093/02,2010-03-22T03:00:00.000000000+0300,2288-Пр/10,5.99700138252e+12
8070,Калия аспарагинат магния аспарагинат,Панангин,раствор для внутривенного введения 10мл -ампулы (5) - упаковки контурные пластиковые (поддоны) - пачки картонные,5.0,108.52,П N013093/01,2011-09-21T04:00:00.000000000+0400,681/31-11,5.99700139152e+12
10631,Калия аспарагинат магния аспарагинат,Панангин,концентрат для приготовления раствора для инфузий 45 2 мг/мл+40мг/мл 10мл - ампулы (5) - упаковки контурные пластиковые - пачки картонные,5.0,108.52,П N013093/01,2012-09-25T04:00:00.000000000+0400,193/20-12,5.99700139152e+12
18516,Калия аспарагинат + Магния аспарагинат,Панангин,таблетки покрытые пленочной оболочкой 50 шт. - флаконы (1) - пачки картонные,50.0,100.83,П N013093/02,2014-11-19T03:00:00.000000000+0300,711/20-14,4.60546900173e+12
20207,Калия аспарагинат+Магния аспарагинат,Панангин Форте,таблетки покрытые пленочной оболочкой 316 мг+280 мг 15 шт. - блистеры (6) - пачки картонные,90.0,647.35,ЛП-002814,2015-04-13T03:00:00.000000000+0300,356/20-15,5.99700136515e+12
20243,Калия аспарагинат+Магния аспарагинат,Панангин Форте,таблетки покрытые пленочной оболочкой 316 мг+280 мг 15 шт. - блистеры (4) - пачки картонные,60.0,431.57,ЛП-002814,2015-04-13T03:00:00.000000000+0300,356/20-15,5.99700136514e+12
20305,Калия аспарагинат+Магния аспарагинат,Панангин Форте,таблетки покрытые пленочной оболочкой 316 мг+280 мг 15 шт. - блистеры (2) - пачки картонные,30.0,215.78,ЛП-002814,2015-04-13T03:00:00.000000000+0300,356/20-15,5.99700136514e+12


In [361]:
max_prices = get_max_prices(data)

In [352]:
serps = load_serps()

In [419]:
%run -n main.py
not_found, no_forms, join = join_forms(serps, max_prices)

In [423]:
print 'Total:', len(serps)
print 'Not found:', len(not_found)
print 'No forms:', len(no_forms)
print 'In join:', len(join)
print

total = 0
matches = 0
no_matches = {}
for (name, title), forms in join.iteritems():
    for (id, form), match in forms.iteritems():
        total += 1
        if match:
            matches += 1
        else:
            no_matches[form] = max_prices[title]
print 'Forms:', total
print 'Matches:', matches
print

for pattern, forms in sorted(no_matches.iteritems()):
    print pattern
    for form, amount in sorted(forms)[:5]:
        print '  ', form, amount
    print

Total: 110
Not found: 6
No forms: 26
In join: 78

Forms: 154
Matches: 107

L-тироксин, таб. 100 мкг
   таблетки 0.1 мг  50 шт. - упаковки ячейковые контурные (2) - пачки картонные 100.0
   таблетки 0.1 мг  50 шт. - упаковки ячейковые контурные - пачки картонные 50.0
   таблетки 50 мкг  50 шт. - упаковки ячейковые контурные - пачки картонные 50.0

Авелокс®, р-р д/инф. 400 мг/250 мл
   раствор для инфузий 1.6 мг/мл  250 мл - флаконы (1) - пачки картонные 1.0
   раствор для инфузий 1.6 мг/мл  250 мл - флаконы - пачки картонные 1.0
   таблетки покрытые оболочкой 400 мг  5 шт. - упаковки ячейковые контурные - пачки картонные 5.0
   таблетки покрытые оболочкой 400 мг  7 шт. - упаковки ячейковые контурные (1) - пачки картонные 7.0
   таблетки покрытые пленочной оболочкой 400 мг  5 шт. - блистеры (1) - пачки картонные 5.0

Альдецин®, спрей назальный 50 мкг/1 доза
   спрей назальный 50 мкг/доза  8.5 г - флаконы с дозирующим устройством и мундштуком  закрытом колпачком - пачки картонные 1.0

Амб

In [428]:
for (name, title), forms in join.iteritems():
    print title
    for (id, pattern), match in forms.iteritems():
        if match:
            print '  ', pattern
            for (form, amount), price in match.iteritems():
                print '    ', amount, price, form
    print

Симвастатин Алкалоид
   Симвастатин алкалоид, таб., покр. пленочной обол., 10 мг
     28.0 178.2 таблетки покрытые пленочной оболочкой 10 мг  7 шт. - упаковки ячейковые контурные (4) - пачки картонные
   Симвастатин алкалоид, таб., покр. пленочной обол., 20 мг
     28.0 248.4 таблетки покрытые пленочной оболочкой 20 мг  14 шт. - упаковки ячейковые контурные (2) - пачки картонные
   Симвастатин алкалоид, таб., покр. пленочной обол., 40 мг
     28.0 376.41 таблетки покрытые пленочной оболочкой 40 мг  14 шт. - упаковки ячейковые контурные (2) - пачки картонные

Амлорус
   Амлорус®, таб. 5 мг
     30.0 61.54 таблетки 5 мг  10 шт. - упаковки ячейковые контурные (3) - пачки картонные
   Амлорус®, таб. 10 мг
     30.0 99.15 таблетки 10 мг  10 шт. - упаковки ячейковые контурные (3) - пачки картонные

КАРВЕДИЛОЛ
   Карведилол, таб. 25 мг
     30.0 239.73 таблетки 25 мг  10 шт. - упаковка контурная ячейковая (3) - пачка картонная

Авелокс
   Авелокс®, таб., покр. пленочной оболочкой, 400 мг
    

In [282]:
%run -n main.py

In [289]:
list(parse_prices(get_prices('avelox', 1958)))

[Авелокс табл.п.о. 400мг n5: 850.0р (А5.  Аптека №362),
 Авелокс таб.п/о плен. 400мг 5: 851.0р (Ригла г.Москва (76, Пятницкая)),
 Авелокс табл.п.о. 400мг n5: 823.0р (А5.  Аптека №651),
 Авелокс табл.п.о. 400мг n5: 787.0р (A5. НОРМА.  Аптека №481),
 Авелокс таб.п/о плен. 400мг 5: 847.0р (Аптечная сеть Оз г.Москва (13, Большая Полянка)),
 Авелокс таб.п/о плен. 400мг 5: 833.0р (Ригла г. Москва),
 Авелокс табл.п.о. 400мг n5: 838.0р (А5.  Аптека №855 ),
 Авелокс табл.п.о. 400мг n5: 851.0р (А5.  Аптека №590),
 Авелокс таб.п/о плен. 400мг 5: 847.0р (Ригла г.Москва (102, Арбат)),
 Авелокс табл.п.о. 400мг n5: 844.0р (А5.  Аптека №956),
 Авелокс 400мг 5 табл.: 797.5р (Софрино г.Москва),
 Авелокс тб п/о 400мг бл n5: 802.0р (ООО &quot;Домашняя медицина&quot;.),
 Авелокс таб п.о 400мг 5: 806.0р (ООО &quot;НЕО-ФАРМ&quot;),
 Авелокс тбл п/о 400мг 5: 770.0р (ООО &quot;Вега&quot;),
 Авелокс тб п/о 400мг бл n5: 770.0р (ООО &quot;Сиджам&quot;),
 Авелокс таб п.о 400мг 5: 807.0р (ООО НЕО-ФАРМ),
 Авелокс, т